# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [32]:
from pyspark.sql import SparkSession

In [33]:
spark = SparkSession.builder.appName('exercise').getOrCreate()

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [34]:
df = spark.read.csv('../Spark_DataFrame_Project_Exercise/walmart_stock.csv',header=True,inferSchema=True)

#### What are the column names?

In [35]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [36]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Print out the first 5 columns.

In [37]:
df.head(5)

[Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date=datetime.datetime(2012, 1, 4, 0, 0), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date=datetime.datetime(2012, 1, 5, 0, 0), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date=datetime.datetime(2012, 1, 6, 0, 0), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date=datetime.datetime(2012, 1, 9, 0, 0), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

#### Use describe() to learn about the DataFrame.

In [41]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [47]:
df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [49]:
from pyspark.sql.functions import format_number

In [52]:
result = df.describe()

In [66]:
result.select('summary', format_number(result['Open'].cast('float'),2).alias('Open'),
                format_number(result['Open'].cast('float'),2).alias('Open'),
                format_number(result['High'].cast('float'),2).alias('High'),
              format_number(result['Low'].cast('float'),2).alias('Low'),
              format_number(result['Close'].cast('float'),2).alias('Close'),
              result['Volume'].cast('int')).alias('Volume').show()

+-------+--------+--------+--------+--------+--------+--------+
|summary|    Open|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.77|    6.74|    6.76| 4519780|
|    min|   56.39|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+--------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [122]:
df2 = df.withColumn("HV Ratio", df['High']/df['Volume'])
df2.select("HV Ratio").show(5)

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
+--------------------+
only showing top 5 rows



#### What day had the Peak High in Price?

In [75]:
df3 = df.orderBy(df['High'].desc()).head(1)
df3[0][0]

datetime.datetime(2015, 1, 13, 0, 0)

#### What is the mean of the Close column?

In [82]:
from pyspark.sql.functions import mean
df.select(format_number(mean(df['Close']),2).alias('AVG Close')).show()

+---------+
|AVG Close|
+---------+
|    72.39|
+---------+



#### What is the max and min of the Volume column?

In [83]:
from pyspark.sql.functions import max, min

In [85]:
df.select(max('Volume').alias('Max'), min('Volume').alias('Min')).show()

+--------+-------+
|     Max|    Min|
+--------+-------+
|80898100|2094900|
+--------+-------+



#### How many days was the Close lower than 60 dollars?

In [107]:
df4 = df.filter(df['Close'] > 90)
df4 = df4.select(format_number('Open', 2).alias("Open Price"),format_number('Close', 2).alias("Close Price"))
df4.select("*").show()

+----------+-----------+
|Open Price|Close Price|
+----------+-----------+
|     89.21|      90.47|
|     89.36|      90.02|
+----------+-----------+



In [111]:
from pyspark.sql.functions import count
result = df.filter(df["Close"] < 60)
result.select(count('*').alias('Total Number')).show()

+------------+
|Total Number|
+------------+
|          81|
+------------+



#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [118]:
high_value = df.filter(df["High"] > 80).count()
total_days = df.count()
result = high_value / total_days * 100
result

9.141494435612083

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [120]:
from pyspark.sql.functions import corr
df.select(corr('High', 'Volume')).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



#### What is the max High per year?

In [143]:
from pyspark.sql.functions import year, month
yeardf = df.withColumn("Year", year(df['Date']))

In [140]:
max_df = yeardf.groupBy("Year").max()
max_df = max_df.orderBy(max_df["max(High)"].desc())
max_df.select("Year", format_number(max_df["max(High)"], 2).alias('High')).show()

+----+-----+
|Year| High|
+----+-----+
|2015|90.97|
|2014|88.09|
|2013|81.37|
|2012|77.60|
|2016|75.19|
+----+-----+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [172]:
from pyspark.sql.functions import avg
add_month = df.withColumn("Month", month(df['Date']))
group_by_month = add_month.groupBy("Month").avg()
order_by_month = group_by_month.orderBy("avg(Month)")
order_by_month.select(add_month['Month'], format_number(order_by_month["avg(Close)"], 2).alias('AVG Close')).show()

+-----+---------+
|Month|AVG Close|
+-----+---------+
|    1|    71.45|
|    2|    71.31|
|    3|    71.78|
|    4|    72.97|
|    5|    72.31|
|    6|    72.50|
|    7|    74.44|
|    8|    73.03|
|    9|    72.18|
|   10|    71.58|
|   11|    72.11|
|   12|    72.85|
+-----+---------+



# Great Job!